# Fit emimTF2N data

EoS: SRK

Parameter T dependence: Order 3 Polynomial

N (total fitting parameters): 8

**Finds an optimal solution, but can't get it to work when plotting - the control volume fails to initialize

## Import Functions

In [ ]:
import idaes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

import sys
sys.path.append('../../../')

from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.opt import TerminationCondition, SolverStatus

from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
# Import unit models from the model library
from idaes.generic_models.unit_models import Flash
# Import degrees of freedom tool
from idaes.core.util.model_statistics import degrees_of_freedom

import scipy.stats as stats

## Load Data

In [ ]:
data = pd.read_csv('../r125_emimtf2n_subset.csv')

In [ ]:
# Model params
params = pd.read_csv('Params/SRK_params_polyTdep.csv',header=None)

In [ ]:
from hfc125_emimtf2n_SRK_polynomial import configuration 

## Apply Parameters + Solve at Each Data Point

In [ ]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [ ]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m.fs.F101.inlet.flow_mol.fix(1)
m.fs.F101.inlet.temperature.fix(283.15)
m.fs.F101.inlet.pressure.fix(100000)
m.fs.F101.inlet.mole_frac_comp[0,'R125'].fix(0.1)

m.fs.properties.SRK_kappa_A['R125','emimTf2N'] = float(params.iloc[1])
m.fs.properties.SRK_kappa_A['emimTf2N','R125'] = float(params.iloc[0])
m.fs.properties.SRK_kappa_B['R125','emimTf2N'] = float(params.iloc[3])
m.fs.properties.SRK_kappa_B['emimTf2N','R125'] = float(params.iloc[2])
m.fs.properties.SRK_kappa_C['R125','emimTf2N'] = float(params.iloc[5])
m.fs.properties.SRK_kappa_C['emimTf2N','R125'] = float(params.iloc[4])
m.fs.properties.SRK_kappa_D['R125','emimTf2N'] = float(params.iloc[7])
m.fs.properties.SRK_kappa_D['emimTf2N','R125'] = float(params.iloc[6])

# Initialize the flash unit
m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)

# Fix the state variables on the state block
# m.fs.F101.inlet.pressure.unfix()
m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].unfix()

R125_x = np.zeros((len(data)))
P = np.zeros((len(data)))
T = np.zeros((len(data)))

m.fs.liq = Param(mutable=True,default=0.040)
m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, "emimTf2N"] == m.fs.liq)

for i in range(len(data)):
    print('i:',i)
    print('Try x_IL:',data["x_emimTf2N"].iloc[i])
    m.fs.liq = data["x_emimTf2N"].iloc[i]
    m.fs.F101.inlet.temperature.fix(float(data["temperature"].iloc[i]))
    m.fs.F101.inlet.pressure.fix(float(data["pressure"].iloc[i]))
    m.fs.F101.inlet.mole_frac_comp[0,'R125'].fix(float(data["x_R125"].iloc[i])+0.1)
    m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].fix(float(1-(data["x_R125"].iloc[i]+0.1)))
    m.fs.F101.vap_outlet.temperature.fix(float(data["temperature"].iloc[i]))
    
    try:
        m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
        
    except:
        print('Initialization Error')
        
    try:

        status = solver.solve(m, tee = False)
        
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):
            print('Feasible/Optimal Solution')
            R125_x[i] = value(m.fs.F101.liq_outlet.mole_frac_comp[0,'R125']) 
            P[i] = value(m.fs.F101.vap_outlet.pressure[0])
            T[i] = data["temperature"].iloc[i]
        else:
            print('Infeasible Solution')
            R125_x[i] = np.nan
            P[i] = np.nan
            T[i] = data["temperature"].iloc[i]
            
    except ValueError:
        R125_x[i] = np.nan
        P[i] = np.nan
        T[i] = data["temperature"].iloc[i]
        print('Solver Error')

In [ ]:
fitted_data = pd.DataFrame(data=[R125_x,P/1e6,T])
fitted_data = fitted_data.T
fitted_data.columns = ['R125_x','P, MPa','T, K']